In [5]:
import os
import uuid
import json
import asyncio
from kaggle_secrets import UserSecretsClient
from typing import List, Dict, Optional 

# --- ADK Imports ---
# ADDED: LoopAgent for iterative refinement
from google.adk.agents import Agent, SequentialAgent, LlmAgent, LoopAgent 
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService 
from google.adk.memory import InMemoryMemoryService
from google.adk.apps.app import App, ResumabilityConfig 
from google.adk.tools import google_search, FunctionTool, AgentTool, preload_memory 
from google.adk.code_executors import BuiltInCodeExecutor 
from google.adk.tools.tool_context import ToolContext 
from google.adk.agents.callback_context import CallbackContext
from google.adk.plugins.base_plugin import BasePlugin
from google.genai import types

# --- Authentication & Setup (Unchanged) ---
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key setup complete.")
except Exception as e:
    print(
        f"Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# Retry Configuration (Unchanged)
retry_config=types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1, http_status_codes=[429, 500, 503, 504] 
)

# --- Memory Service Setup ---

# 1. Initialize the Long-Term Memory Service
memory_service = InMemoryMemoryService()
print("\nLong-Term Memory Service (InMemory) created.")

# 2. Define the Callback to Auto-Save to Memory
async def auto_save_to_memory(callback_context: CallbackContext) -> None:
    """Automatically save session to memory after each agent turn for long-term knowledge."""
    print("--> Memory Callback: Saving session to memory for future recall...")
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

# 3. Define the custom Plugin to wrap the callback
class MemoryPlugin(BasePlugin):
    """A custom Plugin to integrate the after_agent_callback for Memory saving."""
    def __init__(self, memory_saver_function):
        super().__init__(name="MemorySaverPlugin")
        self._memory_saver = memory_saver_function

    async def after_agent_callback(self, *, agent: Agent, callback_context: CallbackContext) -> None:
        """Runs after the agent finishes execution (the turn ends) to save state."""
        await self._memory_saver(callback_context)

# Instantiate the plugin here, before App definition
memory_saver_plugin = MemoryPlugin(auto_save_to_memory) 

# --- 3. Custom LRO Tool with Pause/Resume Logic (Unchanged) ---
def get_farmer_data(
    location: Optional[str] = None, 
    crops: Optional[List[str]] = None, 
    tool_context: ToolContext = None 
) -> dict: 
    """
    Checks for location/crops. If missing, PAUSES and asks the user for input (LRO).
    Returns: Dictionary with watering status or pending message.
    """
    watering_history = {
        "california, usa": False,
        "chennai, india": True, 
        "paris, france": False,
    }
    
    # --- SCENARIO 1: MISSING MANDATORY INPUTS (Trigger LRO Pause) ---
    if (not location or not crops) and (not tool_context or not tool_context.tool_confirmation):
        if tool_context:
            tool_context.request_confirmation(
                hint=f"I am missing the location and/or crop list. Please provide the missing information.",
                payload={"missing_data": True} 
            )
            # Return 'pending' status to the Agent
            return {
                "status": "pending",
                "message": "Missing mandatory data. Requires user input to continue."
            }
        
        raise ValueError("Location and Crops are mandatory inputs.")

    # --- SCENARIO 2: ALL INPUTS ARE PRESENT (Final execution) ---
    loc_key = location.lower().strip()
    return {"status": "complete", "watered_status": watering_history.get(loc_key, False)}

FarmerDataTool = FunctionTool(get_farmer_data) 
print("FarmerDataTool (Custom LRO Tool) created.")


# --- 4. Specialized Agents (Refined for Loop) ---

# Calculation Agent remains the same
calculator_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""PERSONA: You are a specialized calculator that ONLY responds with Python code. TASK: Translate the arithmetic request into a single Python code block that prints the final result to stdout. RULES: 1. Output MUST be ONLY a Python code block. 2. You are PROHIBITED from performing the calculation yourself.""", 
    code_executor=BuiltInCodeExecutor() 
)
print("CalculationAgent created.")

# A. InputProcessorAgent (Unchanged)
input_processor_agent = Agent(
    name="InputProcessorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    PERSONA: You are a dedicated Data Ingestion Agent. Your task is to extract the 'location' and 'crops' from the user's query.
    1. First, call the **'preload_memory' tool** to fetch any relevant facts about the user from past chats (this data will be available in the prompt).
    2. Then, call the **get_farmer_data tool** using the information available in the prompt (from memory or user query). If the tool returns 'pending', tell the user you are waiting for data.
    
    CRITICAL RULE: If the get_farmer_data tool returns a final result in its response, **YOU MUST ONLY OUTPUT the boolean value of the 'watered_status' field**. DO NOT add any conversational text, explanations, or analysis. The next agent expects only the boolean status.
    """,
    tools=[FarmerDataTool, preload_memory],
    output_key="watered_status"
)
print("InputProcessorAgent created with preload_memory.")

# B. ResearchAgent (Refined to read critique)
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="A geospatial and commodity price researcher that generates a structured JSON report.",
    instruction="""
    PERSONA: You are a Geospatial Research Engine. Your goal is to generate a comprehensive JSON research report.
    
    **CRITIQUE STATUS:** You have received a critique: {critique}
    
    - IF the critique is provided and is NOT "APPROVED", you MUST address the feedback and rewrite the entire report.
    - OTHERWISE (initial run or critique is "APPROVED"), proceed to generate the report.
    
    TASK: Use google_search to find and consolidate the results for all data points needed for the final report.
    
    1. Today's exact date and day (current_date).
    2. The 3-day rain chance (as a percentage) (rain_chance_3_day).
    3. The dominant agricultural season (agricultural_season) for the location/crops (search authoritative sources like FAO if possible).
    4. Current Soil Temperature and Evapotranspiration (ET) rate (soil_temp_et) (search NOAA/NASA).
    5. Typical agricultural soil pH range (soil_ph_range).
    6. Price per kg of Urea and DAP in the local market (urea_dap_price).
    
    CRITICAL RULE: If you need to perform ANY arithmetic, you MUST use the CalculationAgent tool.
    
    OUTPUT RULE: Your entire output MUST be a valid, dense JSON object with no external text or markdown.
    """,
    tools=[
        google_search,
        AgentTool(agent=calculator_agent) 
    ],
    output_key="research_report"
)
print("ResearchAgent refined for LoopAgent use.")

# C. Critic Agent (NEW)
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a JSON validation expert. Review the Research Report: {research_report}.
    
    CRITICAL TASK: Check if the report is a single, valid JSON object containing all required fields: current_date, rain_chance_3_day, agricultural_season, soil_temp_et, soil_ph_range, and urea_dap_price.
    
    - If the JSON is valid AND contains ALL required fields, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide a concise, actionable critique stating exactly which field is missing or why the formatting is invalid.
    """,
    output_key="critique", # Stores "APPROVED" or feedback
)
print("CriticAgent created for research validation.")

# D. Research Refinement Loop (NEW)
research_refinement_loop = LoopAgent(
    name="ResearchRefinementLoop",
    sub_agents=[
        critic_agent,    # Runs first to check the state, which holds the output from the last step (or the initial prompt)
        research_agent,  # Runs second to either generate the report or fix it based on critique
    ],
    max_iterations=3, # Prevents infinite loops
)
print("ResearchRefinementLoop (LoopAgent) created.")


# E. AdvisorAgent (Unchanged)
advisor_agent = Agent(
    name="AdvisorAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config), 
    description="A specialist in synthesizing data to provide the final irrigation and fertilizer advice.",
    instruction="""
    PERSONA: You are the Chief Agronomy Advisor.
    INPUTS: Watering Status ({watered_status} - a boolean) and Research Report ({research_report} - a STRING containing a JSON object).
    TASK: Generate a final farmer briefing. **First, you MUST parse the {research_report} string into a usable JSON object.** Then, use the 'CalculationAgent' tool for any arithmetic.
    
    LOGIC (Risk-Based Weighing):
    1. **Primary Risk Check (Overwatering):** If the 'Watering Status' is **True** (recently watered), the decision is **NO**. This is the highest priority rule to prevent root rot.
    2. **Secondary Need Check:** If the 'Watering Status' is **False**, proceed to recommend watering unless the '3-day rain chance' in the parsed Research Report is greater than 50%.
    
    FINAL OUTPUT FORMAT (Strict):
    - Start with the **CURRENT DATE AND DAY** extracted from the Research Report (current_date field).
    - Follow with a clear, bold **WATERING DECISION: YES/NO**.
    - Immediately follow the decision with a section titled **WATERING ANALYSIS** (Brief reason: Explaining how you weighed the risk of overwatering against the current ET rate and rain chance).
    - Conclude with a detailed **Fertilizer Recommendation** (2-3 key points based on crops/season, pH, and cost-effectiveness, and include the result of the CalculationAgent).
    
    CRITICAL RULE: **YOUR FINAL RESPONSE MUST ADHERE TO THIS STRUCTURE AND LOGIC. DO NOT DEVIATE.**
    """,
    output_key="final_advice",
    tools=[AgentTool(agent=calculator_agent)]
)
print("AdvisorAgent confirmed with CalculationAgent tool.")

# --- 5. LRO App & Runner Setup (Final) ---
root_agent = SequentialAgent(
    name="IrrigationAdvisorPipeline",
    description="Orchestrates data collection, external research via refinement loop, and final irrigation advice.",
    sub_agents=[
        input_processor_agent, 
        research_refinement_loop, # <--- FINAL WORKFLOW STEP: Loop Agent inserted
        advisor_agent            
    ]
)

session_service = InMemorySessionService() 

# 1. Wrap the root agent in a resumable App 
shipping_app = App(
    name="irrigation_coordinator",
    root_agent=root_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
    plugins=[memory_saver_plugin], # Plugin passed to App
)

# 2. Use the base Runner 
lro_runner = Runner(
    app=shipping_app, 
    session_service=session_service,
    memory_service=memory_service, # Memory Service is passed to the Runner
)
print("\nFINAL PIPELINE STATUS: Sequential (LRO) -> Loop (Research/Critique) -> Sequential (Advise).")

# --- LRO Helper Functions (Unchanged) ---
def check_for_approval(events):
    """Check if events contain an approval/confirmation request (PAUSE signal)."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    hasattr(part, "function_call")
                    and part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                        "hint": part.function_call.args.get("hint"),
                    }
    return None

def create_approval_response(approval_info, response_text):
    """Creates a FunctionResponse that ADK understands for resumption."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": True, "message": "Input received."} 
    )
    # The new user text is the key to resuming the agent's logic thread
    return types.Content(
        role="user", 
        parts=[
            types.Part(function_response=confirmation_response),
            types.Part(text=response_text)
        ]
    )

Gemini API key setup complete.

Long-Term Memory Service (InMemory) created.
FarmerDataTool (Custom LRO Tool) created.
CalculationAgent created.
InputProcessorAgent created with preload_memory.
ResearchAgent refined for LoopAgent use.
CriticAgent created for research validation.
ResearchRefinementLoop (LoopAgent) created.
AdvisorAgent confirmed with CalculationAgent tool.

FINAL PIPELINE STATUS: Sequential (LRO) -> Loop (Research/Critique) -> Sequential (Advise).


/tmp/ipykernel_47/2506627348.py:238: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [13]:
# Updated LRO Workflow Function for Session Reuse
async def run_lro_workflow_with_session(query: str, session_id: Optional[str] = None):
    user_id = "farmer_user"
    
    # --- MODIFICATION: Reuse existing session or create a new one ---
    if session_id is None:
        session_id = f"session_{uuid.uuid4().hex[:8]}"
        await session_service.create_session(
            app_name="irrigation_coordinator", user_id=user_id, session_id=session_id
        )
        print(f"Created NEW Session: {session_id}")
    else:
        # We rely on the session already existing in the session_service
        print(f"Reusing EXISTING Session: {session_id}")
    # -------------------------------------------------------------------

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []
    final_response_text = ""
    
    print(f"\n{'='*60}\nUser > {query}\n{'='*60}")
    
    # --- STEP 1: Send initial request (PAUSE point) ---
    async for event in lro_runner.run_async(
        user_id=user_id, session_id=session_id, new_message=query_content
    ):
        events.append(event)
        if event.content and hasattr(event.content.parts[0], 'text'):
            print(f"Agent > {event.content.parts[0].text}")
    
    # --- STEP 2: Detect Pause Event ---
    approval_info = check_for_approval(events)

    if approval_info:
        print("\n*** AGENT PAUSED: Missing Mandatory Data (LRO Detected) ***")
        
        # User Interaction: This is where the workflow pauses for interactive input
        # Note: In a real environment, this input() would be an API call waiting for the user response.
        missing_input = input(f"Agent prompt: {approval_info['hint']}\nYour Answer (e.g., 'corn and tomatoes in Paris'): ")
        
        # --- STEP 3: Call Agent AGAIN to RESUME ---
        resume_content = create_approval_response(approval_info, missing_input)
        
        print(f"\n*** RESUMING AGENT with new input: '{missing_input}' ***")
        
        async for event in lro_runner.run_async(
            user_id=user_id,
            session_id=session_id,
            new_message=resume_content, # Send the combined approval/text payload
            invocation_id=approval_info["invocation_id"], # CRITICAL: Resumes thread
        ):
            if event.content and event.content.parts and hasattr(event.content.parts[0], 'text'):
                final_response_text = event.content.parts[0].text
                print(f"Agent > {final_response_text}")
    else:
        # If no pause, the final answer should be in the last event's text content
        if events and events[-1].content and hasattr(events[-1].content.parts[0], 'text'):
            final_response_text = events[-1].content.parts[0].text
            print(f"Agent > {final_response_text}")
        
    print(f"\n{'='*60}")
    print(f"LRO Workflow complete. Session ID: {session_id}")
    
    # IMPORTANT: Return the session_id so the caller can reuse it for the next turn
    return final_response_text, session_id

In [5]:
# --- Execution Test (Will trigger the LRO pause) ---
# This query is intentionally missing the location and crops, forcing the LRO PAUSE.
await run_lro_workflow("What should I do about watering?")



User > What should I do about watering?


/usr/local/lib/python3.11/dist-packages/google/adk/agents/sequential_agent.py:72: UserWarning: [EXPERIMENTAL] SequentialAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  agent_state = SequentialAgentState(current_sub_agent=sub_agent.name)
/usr/local/lib/python3.11/dist-packages/google/adk/utils/feature_decorator.py:87: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  return orig_init(self, *args, **kwargs)


Agent > I can help with that! What is your location and what crops are you growing?
Agent > I need your location and the crops you are growing to provide watering recommendations.
Agent > I can provide watering recommendations, but I need more information. Please tell me:

1.  **Your location:** (e.g., city, state, or zip code)
2.  **The crops you are growing:** (e.g., corn, tomatoes, wheat)

Once I have this information, I can access the relevant data to give you the best advice.

LRO Workflow complete.


''

In [14]:
!rm -rf advisor-agent
!adk create advisor-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY



Agent created in /kaggle/working/advisor-agent:
- .env
- __init__.py
- agent.py



In [15]:
from IPython.core.display import display, HTML 
from jupyter_server.serverapp import list_running_servers 

# Helper function to get proxied URL for ADK Web UI: Day 4
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"
    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")
    baseURL = servers[0]["base_url"]
    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")
    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"
    
    # [cite_start]Styled HTML for the button: Day 4 [cite: 1327-1347]
    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
    <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
    <strong> IMPORTANT: Action Required</strong>
    </div>
    <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
    The ADK web Ul is <strong>not running yet</strong>. You must start it in the next cell.
    <ol style="margin-top: 10px; padding-left: 20px;">
    <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
    <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
    <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
    </ol>
    <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
    </div>
    <a href='{url}' target='_blank' style="
    display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
    text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
    box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
    Open ADK Web UI (after running cell below)
    </a>
    </div>
    """
    display(HTML(styled_html))
    return url_prefix

# Get the URL prefix and display the instruction button
url_prefix = get_adk_proxy_url()

print("\n--- Running ADK Web UI ---")
!adk web --url_prefix {url_prefix} --host 0.0.0.0 --port 8000 --log_level DEBUG



--- Running ADK Web UI ---
/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [139]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For lo